#### Question 3:


For this question, you would need to use the concepts learnt in Graph analytics session, and use datasets trip.csv and station.csv. The two files contain bike sharing data provided by SF Bay Area Portal. Trip.csv file contains following fields:<br><br>
<ul>
<li>	tripId</li>
<li>	Duration</li>
<li>	StartDate</li>
<li>	EndDate</li>
<li>	StartStation</li>
<li>	StartTerminal</li>
<li>	EndDate</li>
<li>	EndStation</li>
<li>	EndTerminal</li>
<li>	BikeID</li>
<li>	SubscriberType</li>
<li>	ZipCode</li>
</ul>
Station.csv file contains following fields:
<ul>
<li>	stationId</li>
<li>	Name</li>
<li>	Lat (Latitude)</li>
<li>	Long (Longitude)</li>
<li>	Dockcount</li>
<li>	Landmark</li>
<li>	Installation</li>
</ul>

Using the two files, please perform the following:
<ol>
<li>	Import the data and create a graph using GraphFrames (Hint: Your graph will have nodes and edges. Nodes here would be individual stations so id field would be name field in station.csv file. Edges would have src and dst so it would Start Station and End Station fields in trip.csv file respectively. You can make use of other fields as properties of nodes and edges). </li>
<li>	Find out number of incoming connections and outgoing connections for each node and print the top 10 nodes.</li>
<li>	Find out which are the most common direct routes that people take and print top 10.</li>
<li>	From the analysis in b, see which are the stations where people most frequently start their trips but do not come back. (Hint: You might have to think of incoming connections as a ratio of outgoing connections). Print top 10 such stations.</li>
<li>	Find all such patterns where any station a is connected to station b, b is connected to c, but c is not directly connected to a.</li>
<li>	 Run a PageRank algorithm to figure out which is the most important station in the entire graph.</li>
</ol>

<b>Amit Kumar Sharma<b><br>
PGID: 71721079

In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
import os
from pyspark.sql import SQLContext,Row
from pyspark.sql.functions import *
from graphframes import *
import pandas as pd

In [3]:
os.chdir('/Users/a5sharma/Documents/ISB/BDM/HWAssignment/Assignment/data')

### Import the data and create a graph using GraphFrames

In [4]:
os.chdir('/Users/a5sharma/Documents/ISB/BDM/HWAssignment/Assignment/data')
trip = sqlContext.read.format("com.databricks.spark.csv").options(header='true', inferschema='true').load('file:///Users/a5sharma/Documents/ISB/BDM/HWAssignment/Assignment/data/trip.csv')
station = sqlContext.read.format("com.databricks.spark.csv").options(header='true', inferschema='true').load('file:///Users/a5sharma/Documents/ISB/BDM/HWAssignment/Assignment/data/station.csv')

In [5]:
trip.printSchema()

root
 |-- Trip ID: integer (nullable = true)
 |-- Duration: integer (nullable = true)
 |-- Start Date: string (nullable = true)
 |-- Start Station: string (nullable = true)
 |-- Start Terminal: integer (nullable = true)
 |-- End Date: string (nullable = true)
 |-- End Station: string (nullable = true)
 |-- End Terminal: integer (nullable = true)
 |-- Bike #: integer (nullable = true)
 |-- Subscriber Type: string (nullable = true)
 |-- Zip Code: string (nullable = true)



In [6]:
station.printSchema()

root
 |-- station_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- dockcount: integer (nullable = true)
 |-- landmark: string (nullable = true)
 |-- installation: string (nullable = true)



In [7]:
tripVertices = station.withColumnRenamed('station_id','id').distinct()
tripVertices.cache()

DataFrame[id: int, name: string, lat: double, long: double, dockcount: int, landmark: string, installation: string]

In [8]:
edges = trip.withColumnRenamed('Trip ID','tripId').withColumnRenamed('Start Date','startDate').withColumnRenamed('Start Station','startStation').withColumnRenamed('Start Terminal','startTerminal').withColumnRenamed('End Station','endStation').withColumnRenamed('End Terminal','endTerminal').withColumnRenamed('Start Date','startDate').withColumnRenamed('End Date','endDate')
edges = edges.withColumnRenamed('Bike #','bikeId').withColumnRenamed('Subscriber Type','subscriberType').withColumnRenamed('Zip Code','zipCode').withColumnRenamed('Duration','duration')

In [9]:
edges.registerTempTable('edges')

In [10]:
tripEdges = sqlContext.sql('select tripId, endDate-startDate as delay, startTerminal as src, endTerminal as dst, startStation as src_station, endStation as dst_station from edges')

In [11]:
tripEdges.cache()

DataFrame[tripId: int, delay: double, src: int, dst: int, src_station: string, dst_station: string]

In [12]:
tripEdges.count()

354152

In [13]:
tripVertices.count()

70

In [14]:
graph = GraphFrame(tripVertices, tripEdges)

### Find out number of incoming connections and outgoing connections for each node and print top 10 nodes? 

In [15]:
inDegree = graph.inDegrees
outDegree = graph.outDegrees

In [16]:
inDegree.sort('inDegree', ascending=False).take(10)

[Row(id=70, inDegree=34810),
 Row(id=69, inDegree=22523),
 Row(id=50, inDegree=17810),
 Row(id=61, inDegree=15463),
 Row(id=65, inDegree=15422),
 Row(id=60, inDegree=15065),
 Row(id=77, inDegree=13916),
 Row(id=74, inDegree=13617),
 Row(id=55, inDegree=12966),
 Row(id=39, inDegree=10239)]

In [17]:
outDegree.sort('outDegree', ascending=False).take(10)

[Row(id=70, outDegree=26304),
 Row(id=69, outDegree=21758),
 Row(id=50, outDegree=17255),
 Row(id=55, outDegree=14436),
 Row(id=60, outDegree=14158),
 Row(id=61, outDegree=14026),
 Row(id=65, outDegree=13752),
 Row(id=74, outDegree=13687),
 Row(id=67, outDegree=11885),
 Row(id=77, outDegree=11431)]

### Find out which are the most common direct routes that people take and print top 10?

In [18]:
topTrips = graph.edges.groupBy("src", "src_station","dst","dst_station").count().sort('count', ascending=False).limit(10)

topTrips.show()

+---+--------------------+---+--------------------+-----+
|src|         src_station|dst|         dst_station|count|
+---+--------------------+---+--------------------+-----+
| 69|San Francisco Cal...| 65|     Townsend at 7th| 3748|
| 50|Harry Bridges Pla...| 60|Embarcadero at Sa...| 3145|
| 61|     2nd at Townsend| 50|Harry Bridges Pla...| 2973|
| 65|     Townsend at 7th| 69|San Francisco Cal...| 2734|
| 50|Harry Bridges Pla...| 61|     2nd at Townsend| 2640|
| 51|Embarcadero at Fo...| 70|San Francisco Cal...| 2439|
| 74|   Steuart at Market| 61|     2nd at Townsend| 2356|
| 60|Embarcadero at Sa...| 74|   Steuart at Market| 2330|
| 65|     Townsend at 7th| 70|San Francisco Cal...| 2192|
| 55|Temporary Transba...| 70|San Francisco Cal...| 2184|
+---+--------------------+---+--------------------+-----+



### From the analysis in b, see which are the stations where people most frequently start their trips but do not come back.

Register inDegree and outDegree as temp table. <br>
Compute ratio of ind.inDegree and outd.outDegree.<br>
If the ratio is high for a station, it means that incoming is more than outgoing for the station and people ended their trip.

If the ratio is low for a station, it means that outgoing is more than incoming for the station and people do not come back to the station.

In [19]:
inDegree.registerTempTable("inDegree")
outDegree.registerTempTable("outDegree")

In [20]:
sqlContext.sql("select ind.id, ind.inDegree/outd.outDegree degreeRatio from inDegree ind, outDegree outd where ind.id == outd.id order by degreeRatio asc limit 20").show()

+---+------------------+
| id|       degreeRatio|
+---+------------------+
| 73|0.5180520570948782|
| 62|0.5909488686085761|
| 71|0.6434241245136186|
| 83|0.6839622641509434|
| 30|0.7413087934560327|
| 56|  0.75726761574351|
| 59|0.7822270981897971|
| 84|0.7857142857142857|
| 58|0.7928849902534113|
| 34|0.8064516129032258|
| 10|0.8064903846153846|
| 66|0.8303267528931245|
| 24|0.8305084745762712|
| 67|0.8599074463609592|
| 13|0.8688915375446961|
| 64|0.8715809483577991|
|  5|0.8718861209964412|
| 11|0.8735213830755232|
| 33|0.8802395209580839|
| 22|0.8927374301675978|
+---+------------------+



### Find all such patterns where any station a is connected to station b, b is connected to c, but c is not directly connected to a.

As there are 354152 records in trip.csv file and it's taking lot's of time to find out motif, that's why I am using sample function to find out motif only for 5298 records.  

In [22]:
edges1 = tripEdges.sample(False,fraction=0.015)
graph = GraphFrame(tripVertices, edges1)
edges1.write.parquet("edges1.parquet")

In [24]:
parquetFile = spark.read.parquet("edges1.parquet")
graph = GraphFrame(tripVertices, parquetFile)

In [25]:
motif = graph.find("(a)-[e]->(b); (b)-[e2]->(c); !(c)-[]->(a)")
motif.take(10)

[Row(a=Row(id=73, name=u'Grant Avenue at Columbus Avenue', lat=37.798522, long=-122.407245, dockcount=15, landmark=u'San Francisco', installation=u'8/21/2013'), e=Row(tripId=433739, delay=None, src=73, dst=55, src_station=u'Grant Avenue at Columbus Avenue', dst_station=u'Temporary Transbay Terminal (Howard at Beale)'), b=Row(id=55, name=u'Temporary Transbay Terminal (Howard at Beale)', lat=37.789756, long=-122.394643, dockcount=23, landmark=u'San Francisco', installation=u'8/20/2013'), e2=Row(tripId=544221, delay=None, src=55, dst=59, src_station=u'Temporary Transbay Terminal (Howard at Beale)', dst_station=u'Golden Gate at Polk'), c=Row(id=59, name=u'Golden Gate at Polk', lat=37.781332, long=-122.418603, dockcount=23, landmark=u'San Francisco', installation=u'8/21/2013')),
 Row(a=Row(id=73, name=u'Grant Avenue at Columbus Avenue', lat=37.798522, long=-122.407245, dockcount=15, landmark=u'San Francisco', installation=u'8/21/2013'), e=Row(tripId=433739, delay=None, src=73, dst=55, src_s

### Run a PageRank algorithm to figure out which is the most important station in the entire graph.

In [31]:
results = graph.pageRank(resetProbability=0.15, tol=0.01)
results.vertices.select("id", "pagerank").show()

+---+-------------------+
| id|           pagerank|
+---+-------------------+
|  2|  4.129655706934019|
|  3|  0.819908658514437|
|  4|  1.758699775396609|
|  5| 0.5076037268076974|
|  6| 1.1815876711444973|
|  7| 0.7580312189324605|
|  8| 0.7049332259364685|
|  9| 1.1997887103034572|
| 10| 0.5332109934725345|
| 11| 0.7890047316577516|
| 12| 0.5681114149192706|
| 13| 0.7471647285281705|
| 14| 0.6105853371191573|
| 16| 0.5423682818176364|
| 21|0.22674683784891417|
| 22|  1.477997766526867|
| 23|0.33535485366350476|
| 24|  2.706746047308352|
| 25| 0.6287951256832548|
| 26| 0.9317499121064922|
+---+-------------------+
only showing top 20 rows



In [32]:
results.vertices.select("id", "pagerank").distinct().sort('pagerank', ascending=False).show()

+---+------------------+
| id|          pagerank|
+---+------------------+
|  2| 4.129655706934019|
| 70|3.2792907133799853|
| 24| 2.706746047308352|
| 28|2.4952781720688595|
| 69|2.2849535341213585|
|  4| 1.758699775396609|
| 50|1.7331238832413067|
| 27|1.6957863598278817|
| 65|1.5730837845213785|
| 74| 1.550020254550833|
| 22| 1.477997766526867|
| 61|1.4311861091004021|
| 55|1.3903177930127997|
| 60|1.3615175825678585|
| 77|  1.29343173927828|
| 67|1.2054547043332389|
|  9|1.1997887103034572|
|  6|1.1815876711444973|
| 34|  1.15426465901257|
| 76| 1.134632297256152|
+---+------------------+
only showing top 20 rows



In [33]:
results.edges.select("src", "dst", "weight").distinct().sort('weight', ascending=False).show()

+---+---+-------------------+
|src|dst|             weight|
+---+---+-------------------+
| 24| 24|                1.0|
| 33| 33|                0.5|
| 26| 22|                0.5|
| 21| 23|                0.5|
| 83| 22|                0.5|
| 26| 26|                0.5|
| 21| 22|                0.5|
| 33| 27|                0.5|
| 23| 23| 0.3333333333333333|
| 23| 24| 0.3333333333333333|
|  5|  2|                0.2|
| 16|  2|                0.2|
| 16|  8|                0.2|
| 16|  7|                0.2|
| 38| 34|                0.2|
|  5| 11|                0.2|
| 38| 38|                0.2|
|  8|  4|0.14285714285714285|
|  8| 16|0.14285714285714285|
|  8|  6|0.14285714285714285|
+---+---+-------------------+
only showing top 20 rows

